<a href="https://colab.research.google.com/github/tanoManzo/hesso/blob/main/hesso_student_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title

## load data

In [6]:
import pandas as pd

df = pd.read_csv('/content/HESSO_student_job.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Category       11 non-null     object
 1   Job            11 non-null     object
 2   Population     11 non-null     int64 
 3   SubJob         10 non-null     object
 4   SubPopulation  10 non-null     object
dtypes: int64(1), object(4)
memory usage: 568.0+ bytes


## pre-process

Get Specific Job List 

In [7]:
df_filter = df.dropna().copy()
df_filter['SubJobList'] = df_filter['SubJob'].apply(lambda x: str(x).split(','))

Get specific population 

In [8]:
def split_and_convert(text):
  return [int(item) for item in text.split(',')]


df_filter['SubPopulationList'] = df_filter['SubPopulation'].apply(lambda x: split_and_convert(x))

## Process

Main Labels (Inner Cicle)

In [9]:
labels_main  = list(set(df['Category'].tolist()))
parents_main = ["" for _ in range(len(labels_main))]
values_main  = [df.groupby('Category')['Population'].sum()[item] for item in labels_main]

Job Labes (Second Circle)

In [10]:
labels_second  = df['Job'].tolist()
parents_second = df['Category'].tolist()
values_second  = df['Population'].tolist()

Detailed Job Position (Outer Circle)

In [11]:
labels_outer  = []
parents_outer = []
values_outer  = []

for idx, row in df_filter.iterrows():
  for job, pop in zip(row['SubJobList'],row['SubPopulationList']):
    labels_outer.append(job)
    parents_outer.append(row['Job'])
    values_outer.append(pop)

Merge First, Second, and Third circles

In [12]:
labels_all  = labels_main  + labels_second  + labels_outer
parents_all = parents_main + parents_second + parents_outer
value_all   = values_main  + values_second  + values_outer

In [27]:
labels_with_perc = []
tot = sum(values_main)

for lab, val in zip(labels_all, value_all):
  text = f'{lab}<br>{round(val/tot*100)}%'
  labels_with_perc.append(text)


## Visualize

In [28]:
import plotly.graph_objects as go

fig = go.Figure(go.Sunburst(
    ids = labels_all,
    labels=labels_with_perc,
    parents=parents_all,
    values=value_all,
    branchvalues="total",
))
fig.update_layout(
    margin=dict(t=0, b=0, r=0, l=0),
    autosize=False,
    width=1100,
    height=1100,
    font=dict(size=18)
)

fig.show()